Import Libraries

In [1]:
import base64
import numpy as np
import cv2
import pika

Set Connection

In [2]:
host = '192.168.56.102' #La mia macchina virtuale
#host = '192.168.1.172'
exchange_name = 'camera'
routing_key_name = 'view'

connection = pika.BlockingConnection(pika.ConnectionParameters(host=host))
channel = connection.channel()

In [3]:
channel.exchange_declare(exchange=exchange_name, exchange_type='topic')

result = channel.queue_declare(queue="monitor_queue", exclusive=True)
queue_name = result.method.queue
channel.queue_bind(exchange = exchange_name,
                   queue = queue_name,
                   routing_key = routing_key_name)

<METHOD(['channel_number=1', 'frame_type=1', 'method=<Queue.BindOk>'])>

Set Action

In [4]:
#Callback
def callback(ch, method, properties, body):
    #Decode the frame
    decodedString = base64.b64decode(body)
    decodedImg = np.frombuffer(decodedString, dtype=np.uint8)
    decodedImg = decodedImg.reshape(480, 640, 3)
    
    #Refresh monitor
    cv2.imshow('Monitor View', decodedImg)
    key = cv2.waitKey(10)
    
    if key % 256 == 27:
        #ESC pressed
        print("Closing...")
        #Close Stuff
        connection.close()
        cv2.destroyAllWindows()

Set Monitor

In [5]:
image = cv2.imread('placeholder.png')
cv2.namedWindow('Monitor View', cv2.WINDOW_AUTOSIZE)
cv2.imshow('Monitor View', image)
key = cv2.waitKey(1)

Consuming

In [6]:
channel.basic_consume(callback,
                      queue=queue_name,
                      no_ack=True)

print('Waiting For Messages')
channel.start_consuming()

Waiting For Messages
Closing...
